In [8]:
import polars as pl

src = "s3://assignment-polars-duckdb-hy536/quazyilx0.txt"

df = pl.read_csv(
    src,
    has_header=False,
    separator=" ",
    new_columns=["date", "time", "fnard_tok", "fnok_tok", "cark_tok", "gnuck_tok"],
)


df.head()

date,time,fnard_tok,fnok_tok,cark_tok,gnuck_tok
str,str,str,str,str,str
"""2000-01-01""","""00:00:03""","""fnard:7""","""fnok:8""","""cark:19""","""gnuck:25"""
"""2000-01-01""","""00:00:08""","""fnard:14""","""fnok:19""","""cark:16""","""gnuck:37"""
"""2000-01-01""","""00:00:17""","""fnard:12""","""fnok:11""","""cark:12""","""gnuck:8"""
"""2000-01-01""","""00:00:22""","""fnard:18""","""fnok:16""","""cark:3""","""gnuck:8"""
"""2000-01-01""","""00:00:32""","""fnard:7""","""fnok:16""","""cark:7""","""gnuck:37"""


In [9]:
df_new = (
    df
    .with_columns(
        pl.concat_str([pl.col("date"), pl.lit(" "), pl.col("time")])
          .str.strptime(pl.Datetime, format="%Y-%m-%d %H:%M:%S", strict=True)
          .alias("date_time"),
        pl.col("fnard_tok").str.split_exact(":", 1).struct.field("field_1").cast(pl.Int32).alias("fnard"),
        pl.col("fnok_tok"). str.split_exact(":", 1).struct.field("field_1").cast(pl.Int32).alias("fnok"),
        pl.col("cark_tok"). str.split_exact(":", 1).struct.field("field_1").cast(pl.Int32).alias("cark"),
        pl.col("gnuck_tok").str.split_exact(":", 1).struct.field("field_1").cast(pl.Int32).alias("gnuck"),
    )
    .select(["date_time","fnard","fnok","cark","gnuck"])
)

df_new.head()

date_time,fnard,fnok,cark,gnuck
datetime[μs],i32,i32,i32,i32
2000-01-01 00:00:03,7,8,19,25
2000-01-01 00:00:08,14,19,16,37
2000-01-01 00:00:17,12,11,12,8
2000-01-01 00:00:22,18,16,3,8
2000-01-01 00:00:32,7,16,7,37


In [10]:
df_new.write_parquet(
    "s3://assignment-polars-duckdb-hy536/quazyilx0.parquet", use_pyarrow=True
) 

In [12]:
anom = df_new.filter(
    (pl.col("fnard")==-1) & 
    (pl.col("fnok")==-1) & 
    (pl.col("cark")==-1) & 
    (pl.col("gnuck")==-1)
)
anom.write_csv("anomalies.csv")